## Imaginary problem statement

McDonald's receives **thousands of customer comments** on their website per day, and many of them are negative. Their corporate employees don't have time to read every single comment, but they do want to read a subset of comments that they are most interested in. In particular, the media has recently portrayed their employees as being rude, and so they want to review comments about **rude service**.

McDonald's has hired you to develop a system that ranks each comment by the **likelihood that it is referring to rude service**. They will use your system to build a "rudeness dashboard" for their corporate employees, so that employees can spend a few minutes each day examining the **most relevant recent comments**.

## Description of the data

Before hiring you, McDonald's used the [CrowdFlower platform](http://www.crowdflower.com/data-for-everyone) to pay humans to **hand-annotate** about 1500 comments with the **type of complaint**. The complaint types are listed below, with the encoding used in the data listed in parentheses:

- Bad Food (BadFood)
- Bad Neighborhood (ScaryMcDs)
- Cost (Cost)
- Dirty Location (Filthy)
- Missing Item (MissingFood)
- Problem with Order (OrderProblem)
- Rude Service (RudeService)
- Slow Service (SlowService)
- None of the above (na)

## Task 1

Read **`mcdonalds.csv`** into a pandas DataFrame and examine it. 

- The **policies_violated** column lists the type of complaint. If there is more than one type, the types are separated by newline characters.
- The **policies_violated:confidence** column lists CrowdFlower's confidence in the judgments of its human annotators for that row (higher is better).
- The **city** column is the McDonald's location.
- The **review** column is the actual text comment.

In [1]:
import pandas as pd
mcdonalds = pd.read_csv('mcdonalds.csv')
mcdonalds.head(2)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\nOrderProblem\nFilthy,1.0\n0.6667\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN


## Task 2

Remove any rows from the DataFrame in which the **policies_violated** column has a **null value**. Check the shape of the DataFrame before and after to confirm that you only removed about 50 rows.

- **Note:** Null values are also known as "missing values", and are encoded in pandas with the special value "NaN". This is distinct from the "na" encoding used by CrowdFlower to denote "None of the above". Rows that contain "na" should **not** be removed.

In [2]:
mcdonalds.shape

(1525, 11)

In [3]:
mcdonalds[pd.notnull(mcdonalds.policies_violated)].shape

(1471, 11)

In [4]:
mcdonalds.isnull().sum()

_unit_id                           0
_golden                            0
_unit_state                        0
_trusted_judgments                 0
_last_judgment_at                  0
policies_violated                 54
policies_violated:confidence      54
city                              87
policies_violated_gold          1525
review                             0
Unnamed: 10                     1525
dtype: int64

In [5]:
mcdonalds = mcdonalds[mcdonalds.policies_violated.notnull()]
mcdonalds.shape

(1471, 11)

In [6]:
mcdonalds.isnull().sum()

_unit_id                           0
_golden                            0
_unit_state                        0
_trusted_judgments                 0
_last_judgment_at                  0
policies_violated                  0
policies_violated:confidence       0
city                              86
policies_violated_gold          1471
review                             0
Unnamed: 10                     1471
dtype: int64

## Task 3

Add a new column to the DataFrame called **"rude"** that is 1 if the **policies_violated** column contains the text "RudeService", and 0 if the **policies_violated** column does not contain "RudeService". The "rude" column is going to be your response variable, so check how many zeros and ones it contains.

In [7]:
mcdonalds['rude'] = mcdonalds.policies_violated.str.contains('RudeService')

In [8]:
mcdonalds.head(2)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10,rude
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\nOrderProblem\nFilthy,1.0\n0.6667\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN,True
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN,True


In [9]:
mcdonalds.rude = mcdonalds.rude.astype(int)

In [10]:
mcdonalds.head(2)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,Unnamed: 10,rude
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\nOrderProblem\nFilthy,1.0\n0.6667\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",NaN,1
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. ŒæI came in at 9:30...,NaN,1


## Task 4

1. Define X (the **review** column) and y (the **rude** column).
2. Split X and y into training and testing sets (using the parameter **`random_state=1`**).
3. Use CountVectorizer (with the **default parameters**) to create document-term matrices from X_train and X_test.

In [11]:
X = mcdonalds.review
y = mcdonalds.rude

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [13]:
print X_train_dtm.shape
print X_test_dtm.shape

(1103, 7300)
(368, 7300)


## Task 5

Fit a Multinomial Naive Bayes model to the training set, calculate the **predicted probabilites** (not the class predictions) for the testing set, and then calculate the **AUC**. Repeat this task using a logistic regression model to see which of the two models achieves a better AUC.

- **Note:** Because McDonald's only cares about ranking the comments by the likelihood that they refer to rude service, **classification accuracy** is not the relevant evaluation metric. **Area Under the Curve (AUC)** is a more useful evaluation metric for this scenario, since it measures the ability of the classifier to assign higher predicted probabilities to positive instances than to negative instances.

In [14]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
y_pred = nb.predict_proba(X_test_dtm)

In [16]:
y_pred[:10, 1]

array([  9.99234643e-01,   2.59242971e-01,   8.57868677e-01,
         2.67695702e-06,   2.68548521e-01,   9.97469782e-01,
         7.40699844e-04,   1.11406133e-09,   1.91914636e-01,
         4.11011564e-01])

In [17]:
from sklearn import metrics
print metrics.roc_auc_score(y_test, y_pred[:, 1])

0.842600540455


### Compare with Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_dtm, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
y_logreg_pred = logreg.predict_proba(X_test_dtm)

In [20]:
y_logreg_pred[:10, 1]

array([  6.25356657e-01,   1.05758935e-01,   7.90119058e-01,
         3.74344600e-02,   4.66568369e-01,   8.91768224e-01,
         2.28933478e-04,   1.48788767e-02,   2.37558400e-01,
         1.08293306e-01])

In [21]:
print metrics.roc_auc_score(y_test, y_logreg_pred[:, 1])

0.823398505802


## Task 6

Using either Naive Bayes or logistic regression (whichever one had a better AUC in the previous step), try **tuning CountVectorizer** and check the testing set **AUC** after each change, and find the set of parameters that increases AUC the most.

In [22]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    
    # create document-term matrices using the vectorizer
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
    
    # print the number of features that were generated
    print 'Features: ', X_train_dtm.shape[1]
    
    # use Multinomial Naive Bayes to predict the star rating
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_prob = nb.predict_proba(X_test_dtm)
    
    # print the accuracy of its predictions
    print 'AUC: ', metrics.roc_auc_score(y_test, y_pred_prob[:, 1])

In [23]:
# don't convert to lowercase
vect = CountVectorizer(lowercase=False)
tokenize_test(vect)

Features:  8742
AUC:  0.840645366396


In [24]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  57936
AUC:  0.819599427754


In [25]:
# remove English stop words
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  7020
AUC:  0.853520902877


In [26]:
# ignore terms that appear in more than 50% of the documents
vect = CountVectorizer(max_df=0.3)
tokenize_test(vect)

Features:  7269
AUC:  0.852376410746


In [27]:
# only keep the top 1000 most frequent terms
vect = CountVectorizer(max_features=3500)
tokenize_test(vect)

Features:  3500
AUC:  0.843506596725


In [28]:
# only keep terms that appear in at least 2 documents
vect = CountVectorizer(min_df=2)
tokenize_test(vect)

Features:  3500
AUC:  0.844587505961


In [29]:
# include 1-grams and 2-grams, and only keep terms that appear in at least 2 documents
vect = CountVectorizer(ngram_range=(1, 2), min_df=3)
tokenize_test(vect)

Features:  8186
AUC:  0.841805754252


In [30]:
vect = CountVectorizer(stop_words='english', max_df=0.3, min_df=2, lowercase=False)
tokenize_test(vect)

Features:  3745
AUC:  0.858384994436


In [31]:
vect = CountVectorizer(stop_words='english', max_df=0.3)
tokenize_test(vect)

Features:  7016
AUC:  0.856223175966


## Task 7 (Challenge)

The **city** column might be predictive of the response, but we are not currently using it as a feature. Let's see whether we can increase the AUC by adding it to the model:

1. Create a new DataFrame column, **review_city**, that concatenates the **review** text with the **city** text. One easy way to combine string columns in pandas is by using the [`Series.str.cat()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.cat.html) method. Make sure to use the **space character** as a separator, as well as replacing **null city values** with a reasonable string value (such as 'na').
2. Redefine X as the **review_city** column, and re-split X and y into training and testing sets.
3. When you run **`tokenize_test()`**, CountVectorizer will simply treat the city as an extra word in the review, and thus it will automatically be included in the model! Check to see whether it increased or decreased the AUC of your **best model**.

In [32]:
mcdonalds['review_city'] = mcdonalds.review.str.cat(mcdonalds.city, sep=' ', na_rep='na')

In [33]:
mcdonalds.review_city[0]

"I'm not a huge mcds lover, but I've been to better ones. This is by far the worst one I've ever been too! It's filthy inside and if you get drive through they completely screw up your order every time! The staff is terribly unfriendly and nobody seems to care. Atlanta"

In [34]:
X = mcdonalds.review_city
y = mcdonalds.rude
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [35]:
vect = CountVectorizer(stop_words='english', max_df=0.3, min_df=2, lowercase=False, binary=False)
tokenize_test(vect)

Features:  3750
AUC:  0.857002066444


In [36]:
# the best model
vect = CountVectorizer(stop_words='english', max_df=0.3, min_df=4)
tokenize_test(vect)

Features:  1739
AUC:  0.864854554125


## Task 8 (Challenge)

The **policies_violated:confidence** column may be useful, since it essentially represents a measurement of the training data quality. Let's see whether we can improve the AUC by only training the model using higher-quality rows!

To accomplish this, your first sub-task is to **calculate the mean confidence score for each row**, and then store those mean scores in a new column. For example, the confidence scores for the first row are `1.0\r\n0.6667\r\n0.6667`, so you should calculate a mean of `0.7778`. Here are the suggested steps:

1. Using the [`Series.str.split()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html) method, convert the **policies_violated:confidence** column into lists of one or more "confidence scores". Save the results as a new DataFrame column called **confidence_list**.
2. Define a function that calculates the mean of a list of numbers, and pass that function to the [`Series.apply()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.apply.html) method of the **confidence_list** column. That will calculate the mean confidence score for each row. Save those scores in a new DataFrame column called **confidence_mean**.

In [37]:
mcdonalds['confidence_list'] = mcdonalds['policies_violated:confidence'].str.split('\n')

In [38]:
def func(l):
    return sum([float(x) for x in l]) / len(l)
mcdonalds['confidence_mean'] = mcdonalds.confidence_list.apply(func)

In [39]:
mcdonalds.confidence_mean[:10]

0    0.77780
1    1.00000
2    1.00000
3    0.66670
4    1.00000
5    0.67775
6    0.65620
7    0.68010
8    0.88890
9    1.00000
Name: confidence_mean, dtype: float64

Your second sub-task is to **remove lower-quality rows from the training set**, and then repeat the model building and evaluation process. Here are the suggested steps:

1. Remove all rows from X_train and y_train that have a **confidence_mean lower than 0.75**. Check their shapes before and after to confirm that you removed about 300 rows.
2. Use the **`tokenize_test()`** function to check whether filtering the training data increased or decreased the AUC of your **best model**.
    - **Hint:** Even though X_train and y_train are separate from the mcd DataFrame, they can still be filtered using a boolean Series generated from mcd because all three objects share the same index.
    - **Note:** It's important that we don't remove any rows from the testing set (X_test and y_test), because the testing set should be representative of the real-world data we will encounter in the future (which will contain both high-quality and low-quality rows).

In [40]:
print X_train.shape
print y_train.shape

(1103L,)
(1103L,)


In [41]:
X_train = X_train[mcdonalds.confidence_mean >= 0.75]
y_train = y_train[mcdonalds.confidence_mean >= 0.75]
print X_train.shape
print y_train.shape

(799L,)
(799L,)


In [42]:
# the best model
vect = CountVectorizer(stop_words='english', max_df=0.3, min_df=4)
tokenize_test(vect)

Features:  1353
AUC:  0.849690033381


## Task 9 (Challenge)

New comments have been submitted to the McDonald's website, and you need to **score them with the likelihood** that they are referring to rude service.

1. Before making predictions on out-of-sample data, it is important to re-train your model on all relevant data using the tuning parameters and preprocessing steps that produced the best AUC above.
    - In other words, X should be defined using either **all rows** or **only those rows with a confidence_mean of at least 0.75**, whichever produced a better AUC above.
    - X should refer to either the **review column** or the **review_city column**, whichever produced a better AUC above.
    - CountVectorizer should be instantiated with the **tuning parameters** that produced the best AUC above.
    - **`train_test_split()`** should not be used during this process.
2. Build a document-term matrix (from X) called **X_dtm**, and examine its shape.
3. Read the new comments stored in **`mcdonalds_new.csv`** into a DataFrame called **new_comments**, and examine it.
4. If your model uses a **review_city** column, create that column in the new_comments DataFrame. (Otherwise, skip this step.)
5. Build a document_term matrix (from the **new_comments** DataFrame) called **new_dtm**, and examine its shape.
6. Train your best model (Naive Bayes or logistic regression) using **X_dtm** and **y**.
7. Predict the "rude probability" for each comment in **new_dtm**, and store the probabilities in an object called **new_pred_prob**.
8. Print the **full text** for each new comment alongside its **"rude probability"**. (You may need to [increase the max_colwidth](http://pandas.pydata.org/pandas-docs/stable/options.html) to see the full text.) Examine the results, and comment on how well you think the model performed!

In [43]:
X = mcdonalds.review_city
y = mcdonalds.rude
vect = CountVectorizer(stop_words='english', max_df=0.3, min_df=4)
X_dtm = vect.fit_transform(X)
X_dtm.shape

(1471, 2104)

In [44]:
new_comments = pd.read_csv('mcdonalds_new.csv')
new_comments.head(2)

,city,review
0,Las Vegas,Went through the drive through and ordered a #...
1,Chicago,Phenomenal experience. Efficient and friendly ...


In [45]:
new_comments.shape

(10, 2)

In [46]:
new_comments['review_city'] = new_comments.review.str.cat(new_comments.city, sep=' ', na_rep='na')

In [47]:
new_comments.review_city[0]

"Went through the drive through and ordered a #10 (cripsy sweet chili chicken wrap) without fries- the lady couldn't understand that I did not want fries and charged me for them anyways. I got the wrong order- a chicken sandwich and a large fries- my boyfriend took it back inside to get the correct order. The gentleman that ordered the chicken sandwich was standing there as well and she took the bag from my bf- glanced at the insides and handed it to the man without even offering to replace. I mean with all the scares about viruses going around... ugh DISGUSTING SERVICE. Then when she gave him the correct order my wrap not only had the sweet chili sauce on it, but the nasty (just not my first choice) ranch dressing on it!!!! I mean seriously... how lazy can you get!!!! I worked at McDonalds in Texas when I was 17 for about 8 months and I guess I was spoiled with good management. This was absolutely ridiculous. I was beyond disappointed. Las Vegas"

In [48]:
new_dtm = vect.transform(new_comments.review_city)
new_dtm.shape

(10, 2104)

In [49]:
nb = MultinomialNB()
nb.fit(X_dtm, y)
new_pred_prob = nb.predict_proba(new_dtm)

In [50]:
new_pred_prob[:, 1]

array([ 0.33533776,  0.00815105,  0.95703848,  0.03903012,  0.39957542,
        0.05921756,  0.27138442,  0.99999236,  0.092585  ,  0.02264287])

In [51]:
pd.set_option('max_colwidth', 4000)
new_comments['rude probability'] = new_pred_prob[:, 1]
new_comments.drop('review', axis=1, inplace=True)
new_comments.sort_values('rude probability', ascending=False)

,city,review_city,rude probability
7,Dallas,My friend and I stopped in to get a late night snack and we were refused service. The store claimed to be 24 hours and the manager was standing right there doing paper work but would not help us. The cashier was only concerned with doing things for the drive thru and said that the manager said he wasn't allowed to help us. We thought it was a joke at first but when realized it wasn't we said goodbye and they just let us leave. I work in a restaurant and this is by far the worst service I have ever seen. I know it was late and maybe they didn't want to be there but it was completely ridiculous. I think the manager should be fired. Dallas,0.999992
2,Los Angeles,Ghetto lady helped me at the drive thru. Very rude and disrespectful to the co workers. Never coming back. Yuck! Los Angeles,0.957038
4,Portland,"I've made at least 3 visits to this particular location just because it's right next to my office building.. and all my experience have been consistently bad. There are a few helpers taking your orders throughout the drive-thru route and they are the worst. They rush you in placing an order and gets impatient once the order gets a tad bit complicated. Don't even bother changing your mind oh NO! They will glare at you and snap at you if you want to change something. I understand its FAST food, but I want my order placed right. Not going back if I can help it. Portland",0.399575
0,Las Vegas,"Went through the drive through and ordered a #10 (cripsy sweet chili chicken wrap) without fries- the lady couldn't understand that I did not want fries and charged me for them anyways. I got the wrong order- a chicken sandwich and a large fries- my boyfriend took it back inside to get the correct order. The gentleman that ordered the chicken sandwich was standing there as well and she took the bag from my bf- glanced at the insides and handed it to the man without even offering to replace. I mean with all the scares about viruses going around... ugh DISGUSTING SERVICE. Then when she gave him the correct order my wrap not only had the sweet chili sauce on it, but the nasty (just not my first choice) ranch dressing on it!!!! I mean seriously... how lazy can you get!!!! I worked at McDonalds in Texas when I was 17 for about 8 months and I guess I was spoiled with good management. This was absolutely ridiculous. I was beyond disappointed. Las Vegas",0.335338
6,Atlanta,"This specific McDonald's is the bar I hold all other fast food joints to now. Been working in this area for 3 years now and gone to this location many times for drive-through pickup. Service is always fast, food comes out right, and the staff is extremely warm and polite. Atlanta",0.271384
8,Cleveland,"Friendly people but completely unable to deliver what was ordered at the drive through. Out of my last 6 orders they got it right 3 times. Incidentally, the billing was always correct - they just could not read the order and deliver. Very frustrating! Cleveland",0.092585
5,Houston,"Why did I revisited this McDonald's again. I needed to use the restroom facilities and the women's bathroom didn't have soap, the floor was wet, the bathroom stink, and the toilets were nasty. This McDonald's is very nasty. Houston",0.059218
3,New York,"Close to my workplace. It was well manged before. Now it's OK. The parking can be tight sometimes. Like all McDonald's, prices are getting expensive. New York",0.039030
9,NaN,"Having visited many McDonald's over the years, I have to say that this one is the most efficient one ever! Even though it is still fast food, the service at the drive-thru is the best. They rarely make a mistake and I never see anyone parked in the drive-thru slots where they bring food out because they don't have it ready. So, if you like McDonald's fast food, it doesn't get any better than this. na",0.022643
1,Chicago,"Phenomenal experience. Efficient and friendly staff. Clean restrooms, good, fast service and bilingual staff. One of the b

In [52]:
# reset the column display width
pd.reset_option('display.max_colwidth')

## Summary:
Correctly predicted good reviews: 9, 1, missed 6  
Correctly predicted rude reviews: 7, 2, 4, 0  
In-between good and bad: 8, 3  
Bad not rude: 5